In [3]:
# -*- coding: utf-8 -*-
"""
Green Space Quality – PyTorch Training Pipeline
===============================================
שחזור הניסוי של המאמר "Green Space Quality Analysis Using Machine Learning Approaches"
אבל עם PyTorch + CUDA במקום TensorFlow, תוך שמירה על:
- חלוקה 70/10/20 (אימון/ולידציה/בדיקה) – סטרטיפית
- ResNet50 Pretrained (ImageNet)
- אוגמנטציות דומות (Albumentations)
- מדדים: Accuracy, Precision/Recall/F1 Macro, Cohen's Kappa, ROC-AUC (מאקרו OVR)
- Grad-CAM לדוגמאות מכל מחלקה

להרצה (ב-CMD/PowerShell, מתוך הסביבה greenspace):



"""

import os
import json
import random
from pathlib import Path
from typing import List, Tuple

import numpy as np
import cv2
import albumentations as A

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    cohen_kappa_score
)

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import models
from tqdm.notebook import tqdm

# -----------------------------
# 🔹 HERE: Define configuration as variables (no argparse needed)
# -----------------------------

class Config:
    root_dir = r"C:\afeca academy\year 2\CVDLP\GreenSpaceQualityProject\OriginalPictures"
    arch = "resnet50"
    batch = 32
    epochs = 50
    seed = 42
    save_dir = "outputs_gsq_torch"
    patience = 5
    gradcam_per_class = 2

# Create args-like object
args = Config()

# 🔹 Classes (ensure they match your folder names)
CLASS_NAMES = ["Healthy", "Dried", "Contaminated"]
NUM_CLASSES = len(CLASS_NAMES)

IMG_SIZE = 224
DEFAULT_SAVE_DIR = args.save_dir


# -----------------------------
# Dataset ו-Augs
# -----------------------------

def build_train_transform(img_size=224):
    """אוגמנטציה בסגנון המאמר."""
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.CLAHE(p=0.5),
        A.Resize(img_size, img_size)
    ])


def build_eval_transform(img_size=224):
    """טרנספורמציה ל-Val/Test – רק resize (אפשר גם normalize)."""
    return A.Compose([
        A.Resize(img_size, img_size)
    ])


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]


class GreenSpaceDataset(Dataset):
    def __init__(self, files: List[str], labels: List[int],
                 transform=None):
        self.files = list(files)
        self.labels = list(labels)
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        path = self.files[idx]
        label = self.labels[idx]
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        if img is None:
            raise ValueError(f"Cannot read image: {path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            augmented = self.transform(image=img)
            img = augmented["image"]

        # albumentations מחזיר np.uint8 HWC, נעשה נרמול ל-0..1 ואז normalize של ImageNet
        img = img.astype(np.float32) / 255.0
        img = (img - IMAGENET_MEAN) / IMAGENET_STD
        # HWC -> CHW
        img = np.transpose(img, (0, 1, 2)) if img.ndim == 3 else img
        img = np.transpose(img, (2, 0, 1))

        img_tensor = torch.from_numpy(img).float()  # float32
        label_tensor = torch.tensor(label, dtype=torch.long)
        return img_tensor, label_tensor


def make_filelist(root_dir: Path, class_names: List[str]) -> Tuple[List[str], List[int]]:
    files = []
    labels = []
    for idx, cname in enumerate(class_names):
        cdir = root_dir / cname
        if not cdir.exists():
            raise FileNotFoundError(f"Missing class folder: {cdir}")
        exts = ("*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tif", "*.tiff", "*.webp")
        for e in exts:
            for p in cdir.glob(e):
                files.append(str(p))
                labels.append(idx)
    return files, labels


def stratified_split(files: List[str], labels: List[int], seed=42):
    """
    חלוקה 70/10/20 – אימון/ולידציה/בדיקה – עם שמירה על יחס מחלקות (stratified).
    """
    files = np.array(files)
    labels = np.array(labels)

    # Train 70%, Temp 30%
    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.30, random_state=seed)
    train_idx, temp_idx = next(sss1.split(files, labels))
    X_train, y_train = files[train_idx], labels[train_idx]
    X_temp, y_temp = files[temp_idx], labels[temp_idx]

    # מתוך Temp – 1/3 Val, 2/3 Test -> 10%/20% מהכל
    val_size = 1.0 / 3.0
    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=(1.0 - val_size), random_state=seed)
    val_idx, test_idx = next(sss2.split(X_temp, y_temp))
    X_val, y_val = X_temp[val_idx], y_temp[val_idx]
    X_test, y_test = X_temp[test_idx], y_temp[test_idx]

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)


# -----------------------------
# בניית המודל (ResNet50)
# -----------------------------

def build_model(arch: str = "resnet50", num_classes: int = NUM_CLASSES, pretrained: bool = True) -> nn.Module:
    arch = arch.lower()
    if arch == "resnet50":
        model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1 if pretrained else None)
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, num_classes)
    else:
        raise ValueError(f"Unsupported arch: {arch}")
    return model


# -----------------------------
# Training / Evaluation
# -----------------------------

def train_one_epoch(model, loader, optimizer, device, criterion):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    pbar = tqdm(loader, desc="Train", leave=False)

    for xb, yb in pbar:
        xb = xb.to(device)
        yb = yb.to(device)

        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * xb.size(0)
        preds = torch.argmax(logits, dim=1)
        all_preds.append(preds.detach().cpu().numpy())
        all_labels.append(yb.detach().cpu().numpy())

        # עדכון טקסט קטן על ה־bar
        current_loss = running_loss / max(1, len(all_labels) * loader.batch_size)
        pbar.set_postfix(loss=f"{current_loss:.4f}")

    epoch_loss = running_loss / len(loader.dataset)
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    acc = accuracy_score(all_labels, all_preds)
    return epoch_loss, acc



@torch.no_grad()
def eval_epoch(model, loader, device, criterion, desc="Eval"):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    all_probs = []

    pbar = tqdm(loader, desc=desc, leave=False)

    for xb, yb in pbar:
        xb = xb.to(device)
        yb = yb.to(device)

        logits = model(xb)
        loss = criterion(logits, yb)

        running_loss += loss.item() * xb.size(0)
        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)

        all_preds.append(preds.cpu().numpy())
        all_labels.append(yb.cpu().numpy())
        all_probs.append(probs.cpu().numpy())

        current_loss = running_loss / max(1, len(all_labels) * loader.batch_size)
        pbar.set_postfix(loss=f"{current_loss:.4f}")

    epoch_loss = running_loss / len(loader.dataset)
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    all_probs = np.concatenate(all_probs)
    acc = accuracy_score(all_labels, all_preds)
    return epoch_loss, acc, all_labels, all_preds, all_probs



def compute_metrics(y_true_idx, y_pred_idx, y_proba, class_names):
    # one-hot ל-ROC-AUC
    num_classes = len(class_names)
    y_true_oh = np.eye(num_classes)[y_true_idx]

    acc = accuracy_score(y_true_idx, y_pred_idx)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_true_idx, y_pred_idx, average="macro", zero_division=0
    )
    kappa = cohen_kappa_score(y_true_idx, y_pred_idx)

    try:
        rocauc = roc_auc_score(y_true_oh, y_proba, multi_class="ovr", average="macro")
    except Exception:
        rocauc = float("nan")

    report = classification_report(y_true_idx, y_pred_idx, target_names=class_names, digits=4, zero_division=0)
    cm = confusion_matrix(y_true_idx, y_pred_idx)

    return {
        "accuracy": acc,
        "precision_macro": prec,
        "recall_macro": rec,
        "f1_macro": f1,
        "kappa": kappa,
        "roc_auc_macro_ovr": rocauc,
        "report": report,
        "confusion_matrix": cm.tolist(),
    }


# -----------------------------
# Grad-CAM פשוט ל-ResNet50
# -----------------------------

def gradcam_resnet50(model, img_tensor, target_class, device, last_layer_name="layer4"):
    """
    img_tensor: tensor בצורת (1,C,H,W) על device
    target_class: אינדקס מחלקה
    """
    model.eval()
    features = None
    gradients = None

    def fwd_hook(module, input, output):
        nonlocal features
        features = output

    def bwd_hook(module, grad_in, grad_out):
        nonlocal gradients
        gradients = grad_out[0]

    # השכבה האחרונה היא model.layer4
    last_layer = dict(model.named_modules())[last_layer_name]
    handle_fwd = last_layer.register_forward_hook(fwd_hook)
    handle_bwd = last_layer.register_backward_hook(bwd_hook)

    # לוודא float32 על ה־device
    img_tensor = img_tensor.to(device=device, dtype=torch.float32)

    logits = model(img_tensor)
    score = logits[0, target_class]
    model.zero_grad()
    score.backward()

    # gradients: (B, C, H, W)
    grads = gradients[0]          # (C,H,W)
    fmap = features[0]            # (C,H,W)

    weights = torch.mean(grads, dim=(1, 2))  # (C,)
    cam = torch.zeros(fmap.shape[1:], dtype=torch.float32, device=device)  # (H,W)
    for w, f in zip(weights, fmap):
        cam += w * f

    cam = torch.relu(cam)
    cam = cam - cam.min()
    cam = cam / (cam.max() + 1e-8)
    cam = cam.detach().cpu().numpy()

    handle_fwd.remove()
    handle_bwd.remove()
    return cam  # HxW, בטווח [0,1]



def save_gradcam_overlay(orig_img_path, cam, out_path, img_size=224):
    """cam בגודל HxW, orig_img_path – תמונה מקורית, נשמור overlay."""
    img = cv2.imread(orig_img_path, cv2.IMREAD_COLOR)
    if img is None:
        return
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_size, img_size))

    heatmap = cv2.resize(cam, (img_size, img_size))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    overlay = cv2.addWeighted((img).astype(np.uint8), 0.6, heatmap, 0.4, 0)
    overlay = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)
    cv2.imwrite(out_path, overlay)


# -----------------------------
# main
# -----------------------------

def main():
    # משתמשים ב-args שמגיע מ-Config בתחילת הקובץ
    global args

    # reproducibility
    torch.manual_seed(args.seed)
    np.random.seed(args.seed)
    random.seed(args.seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    root = Path(args.root_dir)
    assert root.exists(), f"Root dir not found: {root}"

    os.makedirs(args.save_dir, exist_ok=True)

    # 1) איסוף קבצים
    files, labels = make_filelist(root, CLASS_NAMES)
    print(f"נמצאו {len(files)} קבצים.")
    uniq, counts = np.unique(labels, return_counts=True)
    for i, c in zip(uniq, counts):
        print(f"  {CLASS_NAMES[i]}: {c}")

    # 2) חלוקה 70/10/20
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = stratified_split(files, labels, seed=args.seed)
    print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")

    # 3) DataLoaders – num_workers=0 כדי לא להיתקע ב-Notebook על Windows
    train_tf = build_train_transform(IMG_SIZE)
    eval_tf = build_eval_transform(IMG_SIZE)

    train_ds = GreenSpaceDataset(X_train, y_train, transform=train_tf)
    val_ds = GreenSpaceDataset(X_val, y_val, transform=eval_tf)
    test_ds = GreenSpaceDataset(X_test, y_test, transform=eval_tf)

    train_loader = DataLoader(train_ds, batch_size=args.batch, shuffle=True,
                              num_workers=0, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=args.batch, shuffle=False,
                            num_workers=0, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=args.batch, shuffle=False,
                             num_workers=0, pin_memory=True)

    # 4) מודל + Optimizer + Scheduler
    model = build_model(args.arch, NUM_CLASSES, pretrained=True)
    model = model.to(device).float()   # לוודא float32 על ה-GPU/CPU

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="max", factor=0.1, patience=2
    )

    # 5) לולאת אימון עם Early Stopping על val_accuracy
    best_val_acc = 0.0
    best_state = None
    epochs_no_improve = 0

    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": []
    }

    for epoch in range(1, args.epochs + 1):
        print(f"\nEpoch {epoch}/{args.epochs}")
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, device, criterion)
        val_loss, val_acc, _, _, _ = eval_epoch(model, val_loader, device, criterion, desc="Val")

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")

        scheduler.step(val_acc)

        if val_acc > best_val_acc + 1e-4:
            best_val_acc = val_acc
            best_state = model.state_dict()
            epochs_no_improve = 0
            print("  >>> Improvement, saving best model in memory.")
        else:
            epochs_no_improve += 1
            print(f"  No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= args.patience:
            print(f"Early stopping (patience={args.patience})")
            break

    if best_state is not None:
        model.load_state_dict(best_state)

    # 6) הערכה על Test + מדדים
    test_loss, test_acc, y_true, y_pred, y_proba = eval_epoch(
        model, test_loader, device, criterion, desc="Test"
    )
    print(f"\n=== Evaluation on Test Set ===")
    print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

    metrics = compute_metrics(y_true, y_pred, y_proba, CLASS_NAMES)
    print("\nMetrics (summary):")
    for k in ["accuracy", "precision_macro", "recall_macro", "f1_macro", "kappa", "roc_auc_macro_ovr"]:
        v = metrics[k]
        if isinstance(v, float):
            print(f"{k}: {v:.4f}")
        else:
            print(f"{k}: {v}")

    print("\nClassification Report:\n", metrics["report"])
    print("\nConfusion Matrix:\n", np.array(metrics["confusion_matrix"]))

    # 7) שמירת מודל ומדדים
    model_path = os.path.join(args.save_dir, f"model_{args.arch}.pt")
    torch.save(model.state_dict(), model_path)
    print(f"\nSaved model to: {model_path}")

    out_metrics_path = os.path.join(args.save_dir, "test_metrics.json")
    with open(out_metrics_path, "w", encoding="utf-8") as f:
        json.dump(metrics, f, ensure_ascii=False, indent=2)
    print(f"Saved metrics to: {out_metrics_path}")

    hist_path = os.path.join(args.save_dir, "history.json")
    with open(hist_path, "w", encoding="utf-8") as f:
        json.dump(history, f, ensure_ascii=False, indent=2)
    print(f"Saved training history to: {hist_path}")

    # 8) Grad-CAM לכמה דוגמאות מכל מחלקה
    gradcam_dir = os.path.join(args.save_dir, "gradcam")
    os.makedirs(gradcam_dir, exist_ok=True)

    per_class = args.gradcam_per_class
    selected = {i: [] for i in range(NUM_CLASSES)}
    # נבחר מתוך סט הבדיקה
    for path, lbl in zip(X_test, y_test):
        if len(selected[lbl]) < per_class:
            selected[lbl].append(path)
        if all(len(selected[i]) >= per_class for i in range(NUM_CLASSES)):
            break

    print("\nCreating Grad-CAM overlays...")
    model.eval()
    for cls_idx, paths in selected.items():
        for pth in paths:
            # נכין טנזור יחיד
            img = cv2.imread(pth, cv2.IMREAD_COLOR)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

            img_f = img.astype(np.float32) / 255.0
            img_f = (img_f - IMAGENET_MEAN) / IMAGENET_STD
            img_f = np.transpose(img_f, (2, 0, 1))  # CHW

            img_tensor = torch.from_numpy(img_f.astype(np.float32)).unsqueeze(0)

            cam = gradcam_resnet50(model, img_tensor, target_class=cls_idx, device=device)
            base = Path(pth).stem
            out_path = os.path.join(gradcam_dir, f"class{cls_idx}_{base}.png")
            save_gradcam_overlay(pth, cam, out_path, img_size=IMG_SIZE)
            print("  Saved:", out_path)

    # 9) שמירת מידע חלוקה לשחזור
    split_info = {
        "class_names": CLASS_NAMES,
        "train_files": X_train.tolist(),
        "val_files": X_val.tolist(),
        "test_files": X_test.tolist(),
        "arch": args.arch,
        "img_size": IMG_SIZE,
        "batch": args.batch,
        "epochs": args.epochs,
        "seed": args.seed,
    }
    split_path = os.path.join(args.save_dir, "split_config.json")
    with open(split_path, "w", encoding="utf-8") as f:
        json.dump(split_info, f, ensure_ascii=False, indent=2)
    print(f"\nSaved split config to: {split_path}")

    print("\nAll done.")





main()


Using device: cuda
נמצאו 987 קבצים.
  Healthy: 334
  Dried: 322
  Contaminated: 331
Train: 690 | Val: 98 | Test: 199

Epoch 1/50


Train:   0%|          | 0/22 [00:00<?, ?it/s]

Val:   0%|          | 0/4 [00:00<?, ?it/s]

  Train Loss: 0.8316 | Train Acc: 0.6971
  Val   Loss: 1.1527 | Val   Acc: 0.6122
  >>> Improvement, saving best model in memory.

Epoch 2/50


Train:   0%|          | 0/22 [00:00<?, ?it/s]

...

  Train Loss: 0.0430 | Train Acc: 0.9870
  Val   Loss: 0.0428 | Val   Acc: 0.9796
  No improvement for 5 epoch(s).
Early stopping (patience=5)


Test:   0%|          | 0/7 [00:00<?, ?it/s]


=== Evaluation on Test Set ===
Test Loss: 0.0326 | Test Acc: 0.9899

Metrics (summary):
accuracy: 0.9899
precision_macro: 0.9900
recall_macro: 0.9900
f1_macro: 0.9900
kappa: 0.9849
roc_auc_macro_ovr: 0.9997

Classification Report:
               precision    recall  f1-score   support

     Healthy     0.9851    0.9851    0.9851        67
       Dried     1.0000    1.0000    1.0000        65
Contaminated     0.9851    0.9851    0.9851        67

    accuracy                         0.9899       199
   macro avg     0.9900    0.9900    0.9900       199
weighted avg     0.9899    0.9899    0.9899       199


Confusion Matrix:
 [[66  0  1]
 [ 0 65  0]
 [ 1  0 66]]

Saved model to: outputs_gsq_torch\model_resnet50.pt
Saved metrics to: outputs_gsq_torch\test_metrics.json
Saved training history to: outputs_gsq_torch\history.json

Creating Grad-CAM overlays...


c:\Users\User\anaconda3\envs\greenspace\lib\site-packages\torch\nn\modules\module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


  Saved: outputs_gsq_torch\gradcam\class0_img-healthy (8).png
  Saved: outputs_gsq_torch\gradcam\class0_img-healthy (726)-aug.png
  Saved: outputs_gsq_torch\gradcam\class1_img-dried (1).png
  Saved: outputs_gsq_torch\gradcam\class1_img-dried (795).png
  Saved: outputs_gsq_torch\gradcam\class2_img-cont (728).png
  Saved: outputs_gsq_torch\gradcam\class2_img-cont (587).png

Saved split config to: outputs_gsq_torch\split_config.json

All done.
